In [14]:
# import visual libraries and read data
import panel as pn
pn.extension('tabulator', template='material')
import pandas as pd
import hvplot.pandas
import holoviews as hv
import colorcet as cc

dfGroupProd = pd.read_pickle('Q1data/GroupProduction.pkl')
Production = pd.read_pickle('Q1data/CanadaProduction.pkl')
dfPyramidPreds = pd.read_csv('Q1data/predictions.csv')
dfCanadianIntake = pd.read_pickle('Q1data/dfCanadianIntake.pkl')
dfAvail = pd.read_pickle('Q1data/dfAvail.pkl')
dfInnov = pd.read_excel("Q1data/Innovation Funding.xlsx")


## Canadian Diet

In [15]:
widgetAge = dfCanadianIntake.hvplot(kind='box', by='Age Range', y='serving', title='Age vs Serving', hover_cols=['Group'], min_height=1500, min_width=1500, c='serving', cmap=cc.CET_L12, invert=True, legend=False)
widgetEducation = dfCanadianIntake.hvplot(kind='box', by='Education Level', y='serving', title='Education vs Serving', hover_cols=['Group'], min_height=1500, min_width=1500, c='serving', cmap=cc.CET_L12, invert=True, legend=False)
widgetLocality = dfCanadianIntake.hvplot(kind='box', by='Locality', y='serving', title='Locality vs Serving', hover_cols=['Group'], min_height=1500, min_width=1500, c='serving', cmap=cc.CET_L12, invert=True, legend=False)
widgetGender = dfCanadianIntake.hvplot(kind='box', by='Gender', y='serving', title='Gender vs Serving', hover_cols=['Group'], min_height=1500, min_width=1500, c='serving', cmap=cc.CET_L12, invert=True, legend=False)
widgetServings = dfPyramidPreds.hvplot(x="year", groupby='Group', height=450, width=800)

lsDiet = hv.link_selections.instance()
linkDiet = (lsDiet(widgetAge) + lsDiet(widgetEducation) + lsDiet(widgetLocality) + lsDiet(widgetGender)).cols(2)

tabServing = pn.Tabs(dynamic=True)
tabServing.extend([
    ('Canadian Diet', linkDiet),
    ('Diet Predictions', pn.Row(pn.pane.PNG('Q1data/diet effect chart.png'), widgetServings))])

In [16]:
titleServing = pn.panel('''
---
## Canadian serving intake per food group (1990 - 2018)''')
panelServing = pn.Column(titleServing, tabServing)

## Canadian Production

In [17]:
# create two bar plots, add interactivity to view top items produced
widgetGroupProduction = dfGroupProd.hvplot(
    by='Group', y='Value', kind='box', title='Amount Produced Per Food Group (2015 - 2019)', responsive=True, height=450, width=800, invert=True, legend=False)
widgetItemProduction = dfGroupProd.head(60).hvplot(
    by='Item', y='Value', kind='box', title='Amount Produced per Item (2015 - 2019)', responsive=True, height=450, width=800, invert=True, legend=False)
widgetUnitProduction = dfGroupProd.hvplot(
    by='Unit', y='Value', kind='box', title='Amount Produced per Unit of Measure', responsive=True, height=450, width=800, invert=True, legend=False)

lsProd = hv.link_selections.instance()
linkProd = (lsProd(widgetGroupProduction) + lsProd(widgetItemProduction) + lsProd(widgetUnitProduction)).cols(2)

panelProdTitle = pn.panel('''
---
## Candian Food Production (2015-2019)''')
panelProd = pn.Column(panelProdTitle, linkProd)

## Imports and Exports

In [18]:
# ETL using SQL and visualization using PowerBI. Import image into web panel.
tabTrade = pn.Tabs()
tabTrade.extend([
    ('Exports', pn.pane.JPG('Q1data/ExportsMap.jpg')),
    ('Imports', pn.pane.JPG('Q1data/ImportsMap.jpg'))])

panelTradeTitle = pn.panel('''
---
## Canadian Trade Partners Imports/Exports (2017 - 2021)''')
panelTrade = pn.Column(panelTradeTitle, tabTrade)

## Food Availability

In [19]:
widgetAvailDate = dfAvail.groupby(['Supply and disposition', 'REF_DATE', 'VALUE']).sum().reset_index()
widgetAvailDate = widgetAvailDate.hvplot(kind='barh', x='REF_DATE', y='VALUE', groupby='Supply and disposition', min_height=800, min_width=800, responsive=True, invert=True)
widgetAvailSupply = dfAvail.groupby(['Supply and disposition', 'VALUE']).sum().reset_index().hvplot(kind='barh', x='Supply and disposition', y='VALUE', responsive=True, min_height=800, min_width=800, invert=True)
widgetAvailGroup = dfAvail.groupby(['Group', 'VALUE', 'Supply and disposition']).sum().reset_index()
widgetAvailGroup = widgetAvailGroup.hvplot(kind='barh', x='Group', y='VALUE', groupby='Supply and disposition', responsive=True, min_height=800, min_width=800)

tableCommodity = dfAvail[['Commodity', 'Group', 'Supply and disposition', 'VALUE']].groupby(['Commodity', 'Group', 'Supply and disposition']).sum().reset_index().pipe(
    pn.widgets.Tabulator, pagination='remote', page_size=15)

lsAvail = hv.link_selections.instance()
linkAvail = (lsAvail(widgetAvailDate) + lsAvail(widgetAvailGroup)).cols(2)

tabAvail = pn.Tabs(dynamic=True)
tabAvail.extend([
    ('Supply and Disposition', widgetAvailSupply),
    ('Date', linkAvail),
    ('Commodity Table', tableCommodity)])

panelAvailTitle = pn.panel('''
---
## Total Supply and disposition of Food in Canada (1990 - 2020)''')
panelAvail = pn.Column(panelAvailTitle, tabAvail)

## COVID Effects

In [20]:
panelCOVID = pn.Column(pn.pane.PNG(r"C:\Users\proud\DAAN Capstone Proj\Q1data\CPI Capstone project.png"), pn.pane.PNG(r"C:\Users\proud\DAAN Capstone Proj\Q1data\GDP capstone project.png"))

## Innovation

In [21]:
widgetInnovAIFund = dfInnov.hvplot(kind='hist', y='AI Funding', title='AI Funding', responsive=True, min_height=600, min_width=600)
widgetInnovBudget = dfInnov.hvplot(kind='hist', y='Total budget', title='Total Budget', responsive=True, min_height=600, min_width=600)

lsInnov = hv.link_selections.instance()
linkInnov = (lsInnov(widgetInnovBudget) + lsInnov(widgetInnovAIFund))

tabInnov = pn.Tabs(dynamic=True)
tabInnov.extend([
    ('Innovation Talk', pn.pane.PNG('Q1data/innovWordCould.png')),
    ('Innovation Trends', linkInnov),
    ('Innovation Detail', pn.pane.PNG("Q1data/innovDescWordCloud.png"))])
titleInnov = pn.panel('''
---
## Canadian Innovation Projects''')
panelInnov = pn.Column(titleInnov, tabInnov)

## Final Dashboard

In [22]:
pn.state.template.sidebar_width = 350
pn.state.template.title = 'DAAN Capstone Project'
tabQ1 = pn.Tabs(dynamic=True)
tabQ1.extend([ 
    ('Canadian Diet', panelServing), 
    ('Canadian Production', panelProd)])
tabQ2 = pn.Tabs(dynamic=True)
tabQ2.extend([
    ('Canadian Trade Partners', panelTrade),
    ('Food Supply and Disposition', panelAvail)])
tabProj = pn.Tabs(dynamic=True)
tabProj.extend([
    ('Question 1', tabQ1),
    ('Question 2', tabQ2),
    ('Question 4', panelCOVID),
    ('Question 6', tabInnov)])
panelProject = pn.Column(tabProj)

In [23]:
panelProject.servable(title="# How much of Canada’s population can be fed on food grown nationally? How productive is Canada's Food Industry?​")

BokehModel(combine_events=True, render_bundle={'docs_json': {'07076677-1afd-4ebc-8e2d-bc615e547c6b': {'defs': …

Column
    [0] Tabs(dynamic=True)
        [0] Tabs(dynamic=True)
            [0] Column
                [0] Markdown(str)
                [1] Tabs(dynamic=True)
                    [0] HoloViews(Layout, name='Canadian Diet')
                    [1] Row
                        [0] PNG(str)
                        [1] Row
                            [0] HoloViews(DynamicMap)
                            [1] Column
                                [0] WidgetBox
                                    [0] Select(margin=(20, 20, 20, 20), name='Group', options=['Beans, Eggs, ...], value='Beans, Eggs, ..., width=250)
                                [1] VSpacer()
            [1] Column
                [0] Markdown(str)
                [1] HoloViews(Layout)
        [1] Tabs(dynamic=True)
            [0] Column
                [0] Markdown(str)
                [1] Tabs
                    [0] JPG(str)
                    [1] JPG(str)
            [1] Column
                [0] Markdown(str)
                [1] Tabs(dynamic=True)
                    [0] HoloViews(Bars, name='Supply and Disposition')
                    [1] Row(name='Date')
                        [0] HoloViews(Layout, name='Date')
                        [1] Column
                            [0] WidgetBox
                                [0] Select(margin=(20, 20, 20, 20), name='Supply and disposition', options=['Beginning stocks', ...], value='Beginning stocks', width=250)
                            [1] VSpacer()
                    [2] Tabulator(page_size=15, pagination='remote', value=        Commodity         ...)
        [2] Column
            [0] PNG(str)
            [1] PNG(str)
        [3] Tabs(dynamic=True)
            [0] PNG(str)
            [1] HoloViews(Layout, name='Innovation Trends')
            [2] PNG(str)